In [14]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D,Input, Dense, Dropout
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.optimizers import Adadelta
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
from keras.layers import Dropout
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [0]:
def get_vgg16(num_classes, input_shape):
  input_tensor = Input(input_shape)
  base_model = VGG16(input_tensor=input_tensor,include_top = False)
  for layers in base_model.layers:
    layers.trainable = False
  X = base_model.output
  X = GlobalAveragePooling2D(data_format="channels_last")(X)
  X = Dense(num_classes, activation="softmax",kernel_regularizer = keras.regularizers.l2(0.01))(X)
  new_model = Model(base_model.input,X)
  return new_model

In [4]:
model = get_vgg16(num_classes=2,input_shape=(250,250,3))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
train_path = "gdrive/My Drive/chest_xray/train"
test_path ="gdrive/My Drive/chest_xray/test"
val_path ="gdrive/My Drive/chest_xray/val"

In [0]:
train_gen = ImageDataGenerator()

In [7]:
test_data = ImageDataGenerator().flow_from_directory(test_path, target_size=(250,250),batch_size=64)

Found 624 images belonging to 2 classes.


In [8]:
train_data = train_gen.flow_from_directory(train_path, target_size=(250,250), batch_size=64)

Found 5433 images belonging to 2 classes.


In [9]:
val_data = train_gen.flow_from_directory(val_path, target_size=(250,250), batch_size=64)

Found 16 images belonging to 2 classes.


In [0]:
mcp_save = ModelCheckpoint('/content/gdrive/My Drive/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss="categorical_crossentropy",optimizer="adadelta",metrics=["accuracy"])

In [11]:
model.fit_generator(train_data, epochs=100, validation_data=test_data,verbose=1,callbacks=[mcp_save])

Epoch 1/100
85/85 [==============================] - 2041s 24s/step - loss: 0.5459 - acc: 0.8268 - val_loss: 0.8223 - val_acc: 0.7612
Epoch 2/100
85/85 [==============================] - 109s 1s/step - loss: 0.2411 - acc: 0.9251 - val_loss: 0.9010 - val_acc: 0.7644
Epoch 3/100
85/85 [==============================] - 105s 1s/step - loss: 0.1845 - acc: 0.9446 - val_loss: 0.4649 - val_acc: 0.8494
Epoch 4/100
85/85 [==============================] - 105s 1s/step - loss: 0.1619 - acc: 0.9516 - val_loss: 0.6526 - val_acc: 0.8093
Epoch 5/100
85/85 [==============================] - 107s 1s/step - loss: 0.1419 - acc: 0.9582 - val_loss: 0.6896 - val_acc: 0.7997
Epoch 6/100
85/85 [==============================] - 106s 1s/step - loss: 0.1312 - acc: 0.9628 - val_loss: 0.4507 - val_acc: 0.8622
Epoch 7/100
85/85 [==============================] - 110s 1s/step - loss: 0.1149 - acc: 0.9632 - val_loss: 0.9061 - val_acc: 0.7596
Epoch 8/100
85/85 [==============================] - 108s 1s/step - loss: 

In [0]:
model = load_model("/content/gdrive/My Drive/best_model.h5")

In [17]:

model.evaluate_generator(val_data)

[0.03590132296085358, 1.0]

In [18]:
model.evaluate_generator(test_data)

[0.3434002017363524, 0.8862179502462729]